In [64]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn import linear_model,preprocessing
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import os

In [65]:
data_flood = pd.read_csv("/home/studio-lab-user/sagemaker-studiolab-notebooks/DATA/flood_data.csv", sep=",",encoding='latin-1')
data_non_flood = pd.read_csv("/home/studio-lab-user/sagemaker-studiolab-notebooks/DATA/non-flood.csv", sep=",",encoding='latin-1')

In [66]:
data = pd.concat([data_flood, data_non_flood], ignore_index=True).sample(frac=1)[["Rain", "Flood"]]
print(data)
data = data.values

      Rain  Flood
14   226.5      1
61   274.8      1
8    357.3      1
78   310.6      1
143   56.4      0
..     ...    ...
23   238.5      1
7    357.3      1
172  278.7      0
74   356.8      1
88   175.9      1

[196 rows x 2 columns]


In [67]:
X = data[:,0].reshape(-1,1)
y = data[:, 1].reshape(-1,1)
#print(x)
#print(y)

train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size = 0.10, random_state = 18)
train_features = train_features.reshape(-1,1)
test_features = test_features.reshape(-1,1)
train_labels = train_labels.reshape(-1,1)
test_labels = test_labels.reshape(-1,1)

In [68]:
clf = RandomForestClassifier(bootstrap=True,max_depth=66, max_features='sqrt',
                                 min_samples_leaf=1, min_samples_split=2,
                                 n_estimators=452)

clf.fit(train_features, train_labels)

lab_enc = preprocessing.LabelEncoder()
pred_labels_encoded = lab_enc.fit_transform(test_labels)
    
preds = clf.predict(test_features)
confusion_matrix(test_labels, preds)

/tmp/ipykernel_47/1119652961.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(train_features, train_labels)
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[ 4,  3],
       [ 1, 12]])

In [70]:
# Save the model
ckpt_dir = "sagemaker-studiolab-notebooks/Model/Rain_to_flood_model.sav"
pickle.dump(clf, open(ckpt_dir, 'wb'))

NameError: name 'pickle' is not defined

In [ ]:
'''    
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 10, stop = 2000, num = 10)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(1, 110, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]

    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}
    rf = RandomForestClassifier()

    lab_enc = preprocessing.LabelEncoder()
    train_labels_encoded = lab_enc.fit_transform(train_labels)

    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=42, n_jobs = 4)
    rf_random.fit(train_features, train_labels_encoded)
    print (rf_random.best_params_)
'''

### 